In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

Installed package MathNet.Numerics.FSharp version 4.15.0

## Варинат 9

In [ ]:
let a = matrix [
    [-401.64; 200.12]
    [1200.72; -601.76]
]

let b = vector [200.; -600.]

1) Найти решение методом Гаусса

In [ ]:
let solveGauss (matrix: Matrix<float>) (freeVector: Vector<float>) = 
    let n = matrix.RowCount
    let extendedMatrix = matrix |> Matrix.appendCol freeVector
    for k = 0 to n - 1 do
        let temp = extendedMatrix.[k, k]
        if abs temp > 1e-16 then
            for j = k + 1 to n do
                extendedMatrix.[k, j] <- extendedMatrix.[k, j] / temp
        for i = k + 1 to n - 1 do
            let temp = extendedMatrix.[i, k]
            for j = k to n do
                extendedMatrix.[i, j] <- extendedMatrix.[i, j] - extendedMatrix.[k, j] * temp 

    let solution = DenseVector.zero<float> n
    for i = n - 1 downto 0 do
        let mutable sum = 0.
        for j = i + 1 to n - 1 do
            sum <- sum + extendedMatrix.[i, j] * solution.[j]
        solution.[i] <- extendedMatrix.[i, n] - sum

    solution  

In [ ]:
let solution = solveGauss a b
printfn "Решение системы методом Гаусса: %A" solution

2) Преобразовать исходную систему к системе вида x = H * x + g, где H = E − D^(−1) * A, g = D^(−1) * b. Здесь D - диагональная матрица, у которой на диагонали находятся диагональные элементы матрицы A. Вычислить ||H||∞.

In [ ]:
let transform (matrix: Matrix<float>) (freeVector: Vector<float>) = 
    let n = matrix.RowCount 

    // H = E - D^(-1) * A   
    let h = DenseMatrix.init n n (fun i j -> if i = j then 0. else - matrix.[i, j] / matrix.[i, i])
       
    // g = D ^ (-1) * b     
    let g = DenseVector.init n (fun i -> freeVector.[i] / matrix.[i, i])

    h, g

In [ ]:
let (h, g) = transform a b
let norm = h.InfinityNorm()

printfn "Matrix H = %O" h
printfn "Vector g = %O" g
printfn "||H||_inf = %f" norm

Matrix H = 

DenseMatrix 2x2-Double
      0  0,498257
1,99535         0


Vector g = 

DenseVector 2-Double
-0,497958
 0,997075


||H||_inf = 

1.995347

3) Найти априорную оценку того k, при котором ||x∗ − xk||∞ < ε, ε = 0.001

In [ ]:
let epsilon = 0.001

let firstEstimation = DenseVector.zero<float> h.RowCount

let calculatePrioriEstimation k (h: Matrix<float>) (g: Vector<float>) (firstEstiomation: Vector<float>) =
    h.InfinityNorm() ** k * firstEstiomation.InfinityNorm() +  h.InfinityNorm() ** k / (1. - h.InfinityNorm()) * g.InfinityNorm()

let getIterationsCount accuracy (h: Matrix<float>) (g: Vector<float>) (firstEstiomation: Vector<float>) = 
    let mutable iterationsCount = 1.
    while calculatePrioriEstimation iterationsCount h g firstEstiomation >= accuracy do 
        iterationsCount <- iterationsCount + 1.

    int iterationsCount

In [ ]:
printfn "Для получения решения с априорной оценкой %f необходимо %i итераций" 
    <| epsilon
    <| getIterationsCount epsilon h g firstEstimation

Для получения решения с априорной оценкой 

0.001000

 необходимо 

1

 итераций

4) Вычислить решение методом простой итерации с точностью ε = 0.001. Сравнить требуемое фактическое число итераций с априорным значением k. Вывести фактическую погрешность, апостериорную оценку, априорную оценку. Уточнить последнее приближение по Люстернику. Вывести его фактическую погрешность.

In [ ]:
let solveIterative step accuracy (h: Matrix<float>) (g: Vector<float>) = 
    let mutable previousEstimation = firstEstiomation
    let mutable currentEstimation: Vector<float> = step h g previousEstimation
    let mutable iterationCount = 1
    while (previousEstimation - currentEstimation).L2Norm() >= accuracy do
        previousEstimation <- currentEstimation
        currentEstimation <- step h g currentEstimation
        iterationCount <- iterationCount + 1
    
    currentEstimation, iterationCount

let calculatePosteriorEstimation (h: Matrix<float>) (g: Vector<float>) (previousEstiomation: Vector<float>) (currentEstiomation: Vector<float>) = 
    h.InfinityNorm() / (1. - h.InfinityNorm()) * (currentEstiomation - previousEstiomation).InfinityNorm()
    
let solveSimpleIteration (h: Matrix<float>) (g: Vector<float>) =
    let step (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) = 
        h * previousX + g

    solveIterative step epsilon h g

In [ ]:
let getMaxEigenValue (matrix: Matrix<float>) = 
    matrix.Evd().EigenValues.AbsoluteMaximum().Real

let optimizeLusternik (matrix: Matrix<float>) (previousEstiomation: Vector<float>) (currentEstiomation: Vector<float>) = 
    let maxEigen = getMaxEigenValue matrix
    if maxEigen > 1. then
        currentEstiomation
    else 
        previousEstiomation + (1. / (1. - maxEigen)) * (currentEstiomation - previousEstiomation)


In [ ]:
let (solution, iterationCount) = solveSimpleIteration h g
printfn "Решение системы методом простой итерации: %O" solution

Решение системы методом простой итерации: 

DenseVector 2-Double
-0,199422
 0,598267


5) Вычислить решение систем методом Зейделя с точностью ε = 0.001.

In [ ]:
let solveSeidel (h: Matrix<float>) (g: Vector<float>) =
    let step (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) = 
        let x = DenseVector.zero<float> previousX.Count
        for i = 0 to x.Count - 1 do
            x.[i] <- g.[i]
            for j = 0 to i - 2 do
                x.[i] <- x.[i] + h.[i, j] * x.[i]
            for j = i - 1 to x.Count - 1 do
                x.[i] <- x.[i] + h.[i, j] * previousX.[i]
        x

    solveIterative step epsilon h g

In [ ]:
printfn "Solution by Seidel method with accuracy = %f: %O" 
    <| epsilon 
    <| solveSeidel h g

7) Получить решение системы Ax = b методом верхней релаксации с точностью ε = 0.001. В качестве критерия использовать фактическую погрешность.

In [ ]:
let solveUpperRelaxation (h: Matrix<float>) (g: Vector<float>) =
    let q = 2. / (1. + sqrt (1. - (getMaxEigenValue h) ** 2.)) 
    let step (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) = 
        let x = DenseVector.zero<float> previousX.Count
        for i = 0 to x.Count - 1 do
            x.[i] <- g.[i] - previousX.[i]
            for j = 0 to i - 2 do
                x.[i] <- x.[i] + h.[i, j] * x.[i]
            for j = i - 1 to x.Count - 1 do
                x.[i] <- x.[i] + h.[i, j] * previousX.[i]
            x.[i] <- previousX.[i] + q * x.[i]
        x
    
    solveIterative step epsilon h g 
    